```
#!/bin/bash
echo "Start to test the model...."

name="srrawjoint"
dataroot="./SRRAW"

python3 test.py \
    --model srrawjoint  --name $name      --dataset_name srraw  --pre_ispnet_coord False  --gcm_coord True \
    --load_iter 1       --save_imgs True  --calc_metrics True   --gpu_id 0        --visual_full_imgs False \
    --dataroot $dataroot  --scale 4

python3 metrics.py  --name $name --dataroot $dataroot
```

In [1]:
import os
import torch
from options.test_options import TestOptions
from data import create_dataset
from models import create_model
from util.visualizer import Visualizer
from tqdm import tqdm
from util.util import calc_psnr as calc_psnr
import time
import numpy as np
from collections import OrderedDict as odict
from copy import deepcopy


In [2]:

class Bunch(object):
    def __init__(self, adict):
        self.__dict__.update(adict)

opt = {'dataroot': './SRRAW', 
       'dataset_name': 'srraw', 
       'max_dataset_size': float('inf'), 
       'scale': 4, 
       'mode': 'RGB', 
       'imlib': 'cv2', 
       'batch_size': 4, #original 16 
       'patch_size': 224, 
       'shuffle': True, 
       'num_dataloader': 4, 
       'drop_last': True, 
       'gpu_ids': [0], 
       'checkpoints_dir': './ckpt', 
       'verbose': True, 
       'suffix': '', 
       'name': 'srrawjoint', 
       'model': 'srrawjoint', 
       'load_path': '', 
       'load_iter': 1, 
       'gcm_coord': True, 
       'pre_ispnet_coord': False, 
       'chop': False, 
       'init_type': 'default', 
       'init_gain': 0.02, 
       'optimizer': 'Adam', 
       'niter': 1000, 
       'niter_decay': 0, 
       'lr_policy': 'step', 
       'lr_decay_iters': 200, 
       'lr': 0.0001, 
       'load_optimizers': False, 
       'weight_decay': 0, 
       'beta1': 0.9, 
       'beta2': 0.999, 
       'momentum': 0, 
       'alpha': 0.99, 
       'print_freq': 100, 
       'test_every': 1000, 
       'save_epoch_freq': 1, 
       'calc_metrics': True, 
       'save_imgs': True, 
       'visual_full_imgs': False, 
       'isTrain': False, 
       'serial_batches': False, 
       'input_nc': 3, 
       'output_nc': 3
      }

opt = Bunch(opt)

In [3]:
    if not isinstance(opt.load_iter, list):
        load_iters = [opt.load_iter]
    else:
        load_iters = deepcopy(opt.load_iter)

    if not isinstance(opt.dataset_name, list):
        dataset_names = [opt.dataset_name]
    else:
        dataset_names = deepcopy(opt.dataset_name)
    datasets = odict()
    for dataset_name in dataset_names:
        if opt.visual_full_imgs:
            dataset = create_dataset(dataset_name, 'visual', opt)
        else:
            dataset = create_dataset(dataset_name, 'test', opt)
        datasets[dataset_name] = tqdm(dataset)

    for load_iter in load_iters:
        opt.load_iter = load_iter
        model = create_model(opt)
        model.setup(opt)
        model.eval()
        # log_dir = '%s/%s/logs/log_epoch_%d.txt' % (
        #         opt.checkpoints_dir, opt.name, load_iter)
        # os.makedirs(os.path.split(log_dir)[0], exist_ok=True)
        # f = open(log_dir, 'a')

        for dataset_name in dataset_names:
            opt.dataset_name = dataset_name
            tqdm_val = datasets[dataset_name]
            dataset_test = tqdm_val.iterable
            dataset_size_test = len(dataset_test)

            print('='*80)
            print(dataset_name + ' dataset')
            tqdm_val.reset()

            psnr = [0.0] * dataset_size_test

            time_val = 0
            for i, data in enumerate(tqdm_val):
                torch.cuda.empty_cache()
                model.set_input(data)
                torch.cuda.synchronize()
                time_val_start = time.time()
                model.test()
                torch.cuda.synchronize()
                time_val += time.time() - time_val_start
                res = model.get_current_visuals()

                if opt.visual_full_imgs:
                    folder_dir = './ckpt/%s/visual_fullres' % (opt.name)
                    os.makedirs(folder_dir, exist_ok=True)
                    save_dir = '%s/%s.png' % (folder_dir, os.path.basename(data['fname'][0]).split('.')[0])
                    dataset_test.imio.write(np.array(res['data_out'][0].cpu()).astype(np.uint8), save_dir)

                if opt.calc_metrics:
                    psnr[i] = calc_psnr(res['dslr_warp'], res['data_out']*res['dslr_mask']/255.)
                
                if opt.save_imgs:
                    folder_dir = './ckpt/%s/output' % (opt.name)  
                    os.makedirs(folder_dir, exist_ok=True)
                    save_dir = '%s/%s.png' % (folder_dir, os.path.basename(data['fname'][0]).split('.')[0])
                    dataset_test.imio.write(np.array(res['data_out'][0].cpu()).astype(np.uint8), save_dir)

                    folder_dir = './ckpt/%s/warp_gt' % (opt.name)  
                    os.makedirs(folder_dir, exist_ok=True)
                    save_dir = '%s/%s.png' % (folder_dir, os.path.basename(data['fname'][0]).split('.')[0])
                    dataset_test.imio.write(np.array(res['dslr_warp'][0].cpu()).astype(np.uint8), save_dir)

                    folder_dir = './ckpt/%s/warp_gt_mask' % (opt.name) 
                    os.makedirs(folder_dir, exist_ok=True)
                    save_dir = '%s/%s.png' % (folder_dir, os.path.basename(data['fname'][0]).split('.')[0])
                    dataset_test.imio.write(np.array(res['dslr_mask'][0].cpu()).astype(np.uint8), save_dir)

            avg_psnr = '%.2f'%np.mean(psnr)

            # f.write('dataset: %s, PSNR: %s, Time: %.3f sec.\n'
            #         % (dataset_name, avg_psnr, time_val))
            print('Time: %.3f s AVG Time: %.3f ms PSNR: %s\n' % (time_val, time_val/dataset_size_test*1000, avg_psnr))
        #     f.flush()
        #     f.write('\n')
        # f.close()
    for dataset in datasets:
        datasets[dataset].close()


./SRRAW/test/
Starting to load images via multiple imreaders


100%|██████████| 50/50 [00:04<00:00, 12.01it/s]


dataset [srraw(test)] created


  0%|          | 0/50 [00:00<?, ?it/s]

model [SRRAWJOINTModel] was created


/workspace/models/base_model.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(load_path, map_location=self.device)
  0%|          | 0/50 [00:00<

loading the model from ./ckpt/srrawjoint/SRResNet_model_1.pth
All parameters are initialized using [./ckpt/srrawjoint/SRResNet_model_1.pth]
loading the model from ./ckpt/srrawjoint/GCMModel_model_1.pth
All parameters are initialized using [./ckpt/srrawjoint/GCMModel_model_1.pth]
---------- Networks initialized -------------
DataParallel(
  (module): SRResNet(
    (input_stage): Sequential(
      (0): Conv2d(4, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
      (1): PReLU(num_parameters=64)
    )
    (resblock_1): Residual_Block(
      (conv_1): Conv2d(64, 64, kernel_size=(4, 4), stride=(1, 1), bias=False)
      (BatchNorm): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (Prelu): PReLU(num_parameters=64)
      (conv_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (BatchNorm_1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
    )
    (resblock_2): Residual_Bl

 62%|██████▏   | 31/50 [17:34<10:46, 34.02s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.64 GiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Including non-PyTorch memory, this process has 17179869184.00 GiB memory in use. Of the allocated memory 5.98 GiB is allocated by PyTorch, and 1.28 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)